In [2]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import plotly.express as px
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from utils.dataset_generator import CartaDataset

In [3]:
df = pd.read_csv('db\\dados_cartas_yugioh.csv', encoding='utf-8')
df

,id,nome,raca,caminho_imagem
0,11714098,"30,000-Year White Turtle",Aqua,imagens\Aqua\11714098.jpg
1,18318842,Abyss Soldier,Aqua,imagens\Aqua\18318842.jpg
2,88409165,Abyss Warrior,Aqua,imagens\Aqua\88409165.jpg
3,17080584,"Abyssrhine, the Atlantean Spirit",Aqua,imagens\Aqua\17080584.jpg
4,46358784,Advanced Crystal Beast Emerald Tortoise,Aqua,imagens\Aqua\46358784.jpg
...,...,...,...,...
8519,85065943,Saint Azamina,Illusion,imagens\Illusion\85065943.jpg
8520,1528054,Silhouhatte Rabbit,Illusion,imagens\Illusion\1528054.jpg
8521,74150658,Talons of Shurilane,Illusion,imagens\Illusion\74150658.jpg
8522,9275482,UFOLight,Illusion,imagens\Illusion\9275482.jpg


In [ ]:
le = LabelEncoder()
df['raca_codificada'] = le.fit_transform(df['raca'])
classes = le.classes_
classe_to_index = dict(zip(classes, le.transform(classes)))
print("Mapeamento das classes:", classe_to_index)

Mapeamento das classes: {'Aqua': np.int64(0), 'Beast': np.int64(1), 'Beast-Warrior': np.int64(2), 'Dinosaur': np.int64(3), 'Divine-Beast': np.int64(4), 'Dragon': np.int64(5), 'Fairy': np.int64(6), 'Fiend': np.int64(7), 'Fish': np.int64(8), 'Illusion': np.int64(9), 'Insect': np.int64(10), 'Machine': np.int64(11), 'Plant': np.int64(12), 'Psychic': np.int64(13), 'Pyro': np.int64(14), 'Reptile': np.int64(15), 'Rock': np.int64(16), 'Sea Serpent': np.int64(17), 'Spellcaster': np.int64(18), 'Thunder': np.int64(19), 'Warrior': np.int64(20), 'Winged Beast': np.int64(21), 'Wyrm': np.int64(22), 'Zombie': np.int64(23)}


In [5]:
df_train, df_val = train_test_split(df, test_size=0.2, 
                                       stratify=df['raca'], 
                                       random_state=42)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(             
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
        ),
])

print("Tamanho do conjunto de treinamento:", len(df_train))
print("Tamanho do conjunto de validação:", len(df_val))

Tamanho do conjunto de treinamento: 6819
Tamanho do conjunto de validação: 1705


In [6]:
train_dataset = CartaDataset(df_train, transform=transform)
val_dataset = CartaDataset(df_val, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=2)

print(f"Batches de treino: {len(train_loader)}")
print(f"Batches de validação: {len(val_loader)}")

Batches de treino: 214
Batches de validação: 54


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(classes))
model = model.to(device)
criterion = nn.CrossEntropyLoss()

In [8]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(df['raca_codificada']),
    y=df['raca_codificada']
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

# 2. Critério com pesos
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# 3. Liberar treino em todas as camadas
for param in model.parameters():
    param.requires_grad = True

# 4. Otimizador
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 5. Treinamento com validação
num_epochs = 50
best_val_loss = float('inf')
patience = 5
trigger_times = 0

In [9]:
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    running_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss treino: {epoch_loss:.4f}")

    print(f"Validação após a epoch {epoch+1}:")
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    print(classification_report(y_true, y_pred, target_names=classes))
    cm = confusion_matrix(y_true, y_pred)
    px.imshow(cm)

Epoch 1/1 - Loss treino: 3.3727
Validação após a epoch 1:


c:\Users\User\Codes\Data Science\ygo_data\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\Codes\Data Science\ygo_data\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\Codes\Data Science\ygo_data\venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri

               precision    recall  f1-score   support

         Aqua       0.00      0.00      0.00        56
        Beast       0.00      0.00      0.00        83
Beast-Warrior       0.00      0.00      0.00        51
     Dinosaur       0.00      0.00      0.00        29
 Divine-Beast       0.00      0.00      0.00         1
       Dragon       0.11      0.01      0.01       151
        Fairy       0.06      0.06      0.06       111
        Fiend       0.04      0.01      0.01       170
         Fish       0.02      0.72      0.04        29
     Illusion       0.00      0.00      0.00         7
       Insect       0.03      0.17      0.06        53
      Machine       0.07      0.01      0.01       199
        Plant       0.00      0.00      0.00        49
      Psychic       0.00      0.00      0.00        41
         Pyro       0.00      0.00      0.00        31
      Reptile       0.00      0.00      0.00        38
         Rock       0.04      0.02      0.03        52
  Sea Ser

In [10]:
px.imshow(cm)